# Final Project (Pytorch)

Here, basing it off of my homework 4 code, I implement the FinalExamNet with 6 convulational layers, 2 max pooling layers, 2 fully connected layers and a new dropout layer.

In [ ]:
#nn.BatchNorm1d(120)
#nn.Dropout(p=0.5)
#max_pool2d(out, 2)
#.view(out.size(0), -1)
#nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
#nn.Tanh(),
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 11, padding=5)
        self.conv2 = nn.Conv2d(8,16,7,padding=3)
        self.conv3 = nn.Conv2d(16,16,5,padding=2)
        self.conv4 = nn.Conv2d(16,16,5)
        self.conv5 = nn.Conv2d(16,16,5)
        self.conv6 = nn.Conv2d(16,160,5)
        self.fc1   = nn.Linear(160, 160)
        self.fc2   = nn.Linear(160, 10)

        self.dropout = nn.Dropout(0.1) # new dropout layer
        self.dropout = nn.Dropout()

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv5(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv6(out))
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.dropout(out) # new dropout layer added
        out = self.fc2(out)
        return out


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.01
    no_cuda = False
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305735
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.307517
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.300825
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.306260
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.304334
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.303863
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.300376
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.298837
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.299714
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.307365
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.300724
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.300817
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.298781
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.302514
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.304477
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.303573
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.299741
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.303539
Train Epoch: 1 [23040/5000